In [1]:
import pandas as pd
from catboost import CatBoostClassifier
import logging
import pickle
import boto3
from dotenv import load_dotenv
import os
import io
import json

load_dotenv()

AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
S3_SERVICE_NAME = 's3'
S3_ENDPOINT_URL = os.environ.get('S3_ENDPOINT_URL')
BUCKET_NAME = os.environ.get('S3_BUCKET_NAME')

session = boto3.session.Session()

s3 = session.client(
        service_name=S3_SERVICE_NAME,
        endpoint_url=S3_ENDPOINT_URL,
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY
    )

In [2]:
cat_cols = [
    "sexo",
    "ind_nuevo",
    "indext",
    "canal_entrada",
    "cod_prov",
    "ind_actividad_cliente",
    "segmento",
]
num_cols = ["age", "antiguedad", "renta"]
mlc_model = CatBoostClassifier(
    cat_features=cat_cols,
    loss_function="MultiLogloss",
    iterations=10,
    learning_rate=1,
    depth=2,
)
obj_mlc_model = s3.get_object(Bucket=BUCKET_NAME, Key=os.environ.get("KEY_MLC_MODEL"))
mlc_model = mlc_model.load_model(stream=io.BytesIO(obj_mlc_model["Body"].read()))